# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

## Gap detection in DOS

In [ ]:
@jit(nopython=True)
def gap_detector(fermis, IDS, min_IDS_width = 0.001, min_energy_width = 0.1):
    gaps = np.zeros((0,2),dtype=np.float64)
    
    i = 0
    while i <= IDS.size:

        j = 1
        while abs(IDS[i+j]-IDS[i]) < min_IDS_width:
            if j+i == IDS.size-1:
                break
            else:
                j +=1
        else:
            if abs(fermis[i+j]-fermis[i]) > min_energy_width:
                gaps = np.append(gaps,np.array([[fermis[i],fermis[i+j]]]),axis=0)
                i += j
                continue
            else:
                i +=1
                continue
        
        return gaps

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if "n_fermi" in data:
        df = pd.concat([df,data])
    if "n_moments" in data:
        df = pd.concat([df,data])

display(df.set_index('key').sort_index(ascending=False))

## Select data

### skx

In [ ]:
key_spec, key_fermi_l, key_fermi_r = '1700658946', '1700736286', '1700228966' # m=0 s=0 theta_l=0.175 theta_r=0.377

## Import file and extract data

In [ ]:
# parameter
size_l,  size_r  = df.set_index('key').at[key_fermi_l,'system_sizes'], df.set_index('key').at[key_fermi_r,'system_sizes']
theta_l, theta_r = df.set_index('key').at[key_fermi_l,'q'],            df.set_index('key').at[key_fermi_r,'q']

sys_size         = df.set_index('key').at[key_spec,'system_sizes']
t,       m       = df.set_index('key').at[key_spec,'t'],               df.set_index('key').at[key_spec,'m']
shift,   mag     = df.set_index('key').at[key_spec,'shift'],           df.set_index('key').at[key_spec,'mag']

n_energies, n_moments, n_random_states = df.set_index('key').at[key_spec,'n_energies'], df.set_index('key').at[key_spec,'n_moments'], df.set_index('key').at[key_spec,'n_random_states'],
scale, epsilon = 12, 0.01,

n_fermi_l, min_fermi_l, max_fermi_l = df.set_index('key').at[key_fermi_l,'n_fermi'],  df.set_index('key').at[key_fermi_l,'min_fermi'], df.set_index('key').at[key_fermi_l,'max_fermi']
n_fermi_r, min_fermi_r, max_fermi_r = df.set_index('key').at[key_fermi_r,'n_fermi'],  df.set_index('key').at[key_fermi_r,'min_fermi'], df.set_index('key').at[key_fermi_r,'max_fermi']

# Data
outdir_fermi_l = './data/' + key_fermi_l
outdir_fermi_r = './data/' + key_fermi_r
outdir_spec    = './data/' + key_spec

# DOS
fermis_l = np.linspace(min_fermi_l,max_fermi_l,n_fermi_l)
fermis_r = np.linspace(min_fermi_r,max_fermi_r,n_fermi_r)

spectrum_l = np.load(outdir_fermi_l+"/spectrum.npy")
spectrum_r = np.load(outdir_fermi_r+"/spectrum.npy")

# Chern numbers
ch_0_l = np.array([])
for f in fermis_l:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_l])
    ch_0_l = np.append(ch_0_l,[sum(occ)/size_l**2])
    
ch_tau1tau2_l     = np.load(outdir_fermi_l+"/ch_tau1tau2.npy")
ch_u1u2_l         = np.load(outdir_fermi_l+"/ch_u1u2.npy")
ch_tau1tau2u1u2_l = np.load(outdir_fermi_l+"/ch_tau1tau2u1u2.npy")

ch_0_r = np.array([])
for f in fermis_r:
    occ = np.array([1 if energy <= f else 0 for energy in spectrum_r])
    ch_0_r = np.append(ch_0_r,[sum(occ)/size_r**2])
    
ch_tau1tau2_r     = np.load(outdir_fermi_r+"/ch_tau1tau2.npy")
ch_u1u2_r         = np.load(outdir_fermi_r+"/ch_u1u2.npy")
ch_tau1tau2u1u2_r = np.load(outdir_fermi_r+"/ch_tau1tau2u1u2.npy")
    
# data
qs = np.array([i/sys_size for i in range(1,math.floor(sys_size/2))])

emesh = scale*np.linspace(-1, 1, n_energies) * (1-epsilon)

n_q = len(qs)
dos = []
for i in range(n_q):
    dos.append( np.load('./data/'+key_spec+'/dos_'+str(i).zfill(4)+'.npy') )
dos = np.array(dos)

Emin = np.amin(emesh)
Emax = np.amax(emesh)

phimin = np.amin(qs)
phimax = np.amax(qs)

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

tfs    = 24 #title font size
lfs    = 24 #label font size
fs     = 24 #font size
cbarfs = 18 #colorbar font size
afs    = 24 #annotation font size
gfs    = 18 #gap font size

# scatter size
s = 2

linthresh = 0.07 # The range within which the plot is linear
linscale = 1   # The factor by which data smaller than `linthresh` is scaled.
norm = SymLogNorm(linthresh=linthresh, linscale=linscale)

# DoS + Chern

In [ ]:
gaps_l = gap_detector(fermis_l,ch_0_l,0.001,0.1)
gaps_r = gap_detector(fermis_r,ch_0_r,0.001,0.1)
print(gaps_l.shape,gaps_r.shape)
# print(gaps_l)
# print([sum(g)/2 for g in gaps_l])

In [ ]:
# x_ticks0 = np.arange(0, 3, 1)
x_ticks1 = np.arange(0, 2, 1)
x_ticks2 = np.arange(-10, 15, 5)
x_ticks3 = np.arange(-10, 5, 5)
x_ticks4 = np.arange(0.1, 0.5, 0.1)  
# x_ticks5 = np.arange(0, 3, 1)
x_ticks6 = np.arange(0, 2, 1)
x_ticks7 = np.arange(-2, 3, 2)
x_ticks8 = np.arange(-5, 10, 5)
y_ticks = np.arange(-10, 8, 2)
    
fig = plt.figure()
gs = fig.add_gridspec(1,32, hspace=0,wspace=0)
# ax0 = plt.subplot(gs.new_subplotspec((0, 0), colspan=2))
ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=2))
ax2 = plt.subplot(gs.new_subplotspec((0, 2), colspan=6),sharey=ax1)
ax3 = plt.subplot(gs.new_subplotspec((0, 8), colspan=4),sharey=ax1)
ax4 = plt.subplot(gs.new_subplotspec((0, 12), colspan=8),sharey=ax1)
# ax5 = plt.subplot(gs.new_subplotspec((0, 18), colspan=2),sharey=ax0)
ax6 = plt.subplot(gs.new_subplotspec((0, 20), colspan=2),sharey=ax1)
ax7 = plt.subplot(gs.new_subplotspec((0, 22), colspan=5),sharey=ax1)
ax8 = plt.subplot(gs.new_subplotspec((0, 27), colspan=5),sharey=ax1)

# ax0.scatter(ch_0_l,fermis_l,s)
# ax0.set_xticks(x_ticks0)
# ax0.xaxis.set_minor_locator(MultipleLocator(0.2))
# ax0.set_xlim([2.05,-0.05])
# ax0.set_title(r'IDS',fontsize=tfs,y=1.01)

ax1.scatter(ch_u1u2_l.real,fermis_l,s)
ax1.set_xticks(x_ticks1)
ax1.xaxis.set_minor_locator(MultipleLocator(0.5))
ax1.grid(True,which='minor',color='lightgray',axis='x')
ax1.set_xlim([-1,2])
ax1.set_ylabel(r'$E_F$',labelpad=-20.0,fontsize=lfs)
ax1.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs,y=1.005)

ax2.scatter(ch_tau1tau2_l.real,fermis_l,s)
ax2.set_xticks(x_ticks2)
ax2.xaxis.set_minor_locator(MultipleLocator(1))
ax2.grid(True,which='minor',color='lightgray',axis='x')
ax2.set_xlim([-12,14])
ax2.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs,y=1.005)

ax3.scatter(ch_tau1tau2u1u2_l.real,fermis_l,s)
ax3.set_xticks(x_ticks3)
ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax3.grid(True,which='minor',color='lightgray',axis='x')
ax3.set_xlim([-15,5])
ax3.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs,y=1.005)

ax4.imshow(dos.T, aspect='auto',norm=norm, extent=(phimin,phimax, Emin, Emax), interpolation='gaussian', origin = 'lower', resample=True,cmap='Blues');
ax4.set_xticks(x_ticks4)
ax4.set_xlim([phimin,phimax])
ax4.xaxis.set_minor_locator(MultipleLocator(0.05))
ax4.set_title(r"$\text{DOS}$",fontsize=tfs,y=1.005)
ax4.set_xlabel(r"$\vartheta$",fontsize=lfs)

ax4.axvline(x = theta_l, color = 'red', label = r'$\theta_l$')
ax4.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_l*size_l,size_l),(theta_l+0.01,-0.6),color='red',fontsize=afs)
ax4.axvline(x = theta_r, color = 'green', label = r'$\theta_r$')
ax4.annotate(r'$\vartheta=\frac{{ {:.0f} }}{{ {:.0f} }}$'.format(theta_r*size_r,size_r),(theta_r-0.11,-2.2),color='green',fontsize=afs)

ax4.annotate(r'$U-1$',(0.005,7.4),color='black',fontsize=gfs)
ax4.annotate(r'$U-2$',(0.005,6.9),color='black',fontsize=gfs)

ax4.annotate(r'$U1$',(0.01,0.06),color='black',fontsize=gfs)
ax4.annotate(r'$U2$',(0.05,0.7),color='black',fontsize=gfs)
ax4.annotate(r'$U3$',(0.01,1.35),color='black',fontsize=gfs)
ax4.annotate(r'$U4$',(0.05,1.8),color='black',fontsize=gfs)
ax4.annotate(r'$U5$',(0.01,2.3),color='black',fontsize=gfs)
ax4.annotate(r'$U6$',(0.05,2.85),color='black',fontsize=gfs)
ax4.annotate(r'$U7$',(0.01,3.2),color='black',fontsize=gfs)
ax4.annotate(r'$U8$',(0.05,3.55),color='black',fontsize=gfs)
ax4.annotate(r'$U9$',(0.01,3.95),color='black',fontsize=gfs)
ax4.annotate(r'$U10$',(0.045,4.3),color='black',fontsize=gfs)

ax4.annotate(r'$U1$',(0.455,3.1),color='black',fontsize=gfs)
ax4.annotate(r'$U2$',(0.455,4.3),color='black',fontsize=gfs)

ax4.annotate(r'$L-1$',(0.005,-2.6),color='black',fontsize=gfs)

ax4.annotate(r'$L1$',(0.01,-10.05),color='black',fontsize=gfs)
ax4.annotate(r'$L2$',(0.01,-9.6),color='black',fontsize=gfs)

ax4.annotate(r'$L2$',(0.455,-6.7),color='black',fontsize=gfs)

# ax5.scatter(ch_0_r,fermis_r,s)
# ax5.set_xticks(x_ticks5)
# ax5.xaxis.set_minor_locator(MultipleLocator(0.2))
# ax5.set_xlim([2.05,-0.05])
# ax5.set_title(r'IDS',fontsize=tfs,y=1.01)

ax6.scatter(ch_u1u2_r.real,fermis_r,s)
ax6.set_xticks(x_ticks6)
ax6.xaxis.set_minor_locator(MultipleLocator(0.5))
ax6.grid(True,which='minor',color='lightgray',axis='x')
ax6.set_xlim([-1,2])
ax6.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs,y=1.005)

ax7.scatter(ch_tau1tau2_r.real,fermis_r,s)
ax7.set_xticks(x_ticks7)
ax7.xaxis.set_minor_locator(MultipleLocator(1))
ax7.grid(True,which='minor',color='lightgray',axis='x')
ax7.set_xlim([-4,3])
ax7.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs,y=1.005)

ax8.scatter(ch_tau1tau2u1u2_r.real,fermis_r,s)
ax8.set_xticks(x_ticks8)
ax8.xaxis.set_minor_locator(MultipleLocator(1))
ax8.grid(True,which='minor',color='lightgray',axis='x')
ax8.set_xlim([-8,8])
ax8.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs,y=1.005)
ax8.set_ylabel(r'$E_F$',fontsize=lfs)
ax8.yaxis.set_label_position("right")
ax8.get_yaxis().set_visible(False)

for ax in fig.get_axes():
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_yticks(y_ticks)
    ax.set_ylim([-11,8])
    
for ax in fig.get_axes()[0:3]:
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_l:
        ax.fill_between(np.arange(-15,15,0.01),g[0],g[1], facecolor='gray', alpha=0.5)
        
for g in gaps_l:
    ax4.fill_between(np.arange(0.0,theta_l,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
for g in gaps_r:
    ax4.fill_between(np.arange(theta_r,0.51,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
    
for ax in fig.get_axes()[4:7]:
    ax.yaxis.tick_right()
    ax.grid(True,color='gray',axis='x')
    #ax.grid(True,which='minor',color='lightgray')
    
    for g in gaps_r:
        ax.fill_between(np.arange(-15,15,0.001),g[0],g[1], facecolor='gray', alpha=0.5)
        
for ax in fig.get_axes()[1:]:
    ax.yaxis.tick_right()

plt.savefig("./Plots/Chernhierarchy_kpm_0shift0mag.png", dpi=300, bbox_inches = 'tight')
plt.savefig("./Plots/lowres/Chernhierarchy_kpm_0shift0mag.png", dpi=100, bbox_inches = 'tight')